1. Gurobipy: [Column](https://www.gurobi.com/documentation/11.0/refman/py_column.html)
2. Gurobipy: [multidict](https://www.gurobi.com/documentation/11.0/refman/py_multidict.html)
3. Gurobipy: [getAttr](https://www.gurobi.com/documentation/11.0/refman/py_model_getattr.html)

In [9]:
#  ============= NOT USING COLUMN GENERATION VERSION ===============
import random
import gurobipy as gp
from gurobipy import GRB
random.seed(42)
# Types  = [3, 7, 9, 16]          # 需求长度
# Demand = [25, 30, 14,8]          # 需求的量
Types  = [] # 需求长度
Demand = [] # 需求的量
num = 100
for i in range(num):
    Types.append(random.randint(2,16))
    Demand.append(random.randint(9,20))
L = 20

K = 0  #设置木材长度数量上限
for i in range(len(Types)):
    K  += Demand[i]//(L//Types[i]) + 2
print(K)
x = {}
y = {}
m = gp.Model("CSP_COMPACT")
for i in range(len(Types)):
    for k in range(K):
        name_x = "x_" + str(i) + "_" + str(k)
        x[i,k] = m.addVar(vtype= GRB.INTEGER,name=name_x)
for k in range(K):
    name_y = "y_" + str(k)
    y[k] = m.addVar(vtype=GRB.BINARY, name=name_y)

m.setObjective(gp.quicksum(y[k] for k in range(K)),GRB.MINIMIZE)
#约束1
for i in range(len(Types)):
    m.addConstr(gp.quicksum(x[i,k] for k in range(K)) >= Demand[i],\
    name = f"c1_i{i}")
#约束2
for k in range(K):
    m.addConstr(gp.quicksum(Types[i]*x[i,k] \
    for i in range(len(Types)))<=L*y[k],name=f"c2_k{k}")
m.setParam(GRB.Param.TimeLimit,600)
m.optimize()
print(m.objval)

1046
Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1146 rows, 105646 columns and 210246 nonzeros
Model fingerprint: 0x955c9d34
Variable types: 0 continuous, 105646 integer (1046 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+00, 2e+01]
Presolve time: 0.27s
Presolved: 1146 rows, 105646 columns, 210246 nonzeros
Variable types: 0 continuous, 105646 integer (42886 binary)
Found heuristic solution: objective 669.0000000

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 6.480000e+02, 104602 iterations, 0.20 seconds (0.21 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   

In [9]:
# ===================== USING COLUMN GENERATION ALGORITHM ====================
import numpy as np



class MasterProblem:
    def __init__(self):
        self.model = gp.Model("master")
        self.vars = None
        self.constrs = None
        self.objCoef = None
        
    def setup(self, patterns, demand):
        num_patterns = len(patterns)
        self.vars = self.model.addVars(num_patterns, obj=self.objCoef, name="Pattern")
        self.constrs = self.model.addConstrs((gp.quicksum(patterns[pattern][piece]*self.vars[pattern]
                                                          for pattern in range(num_patterns))
                                              >= demand[piece] for piece in demand.keys()),
                                             name="Demand")
        self.model.modelSense = GRB.MINIMIZE
        self.model.params.outputFlag = 0
        self.model.update()
        
    def update(self, pattern, index, objcoef):
        new_col = gp.Column(coeffs=pattern, constrs=self.constrs.values())
        self.vars[index] = self.model.addVar(obj=objcoef, column=new_col,
                                             name=f"Pattern[{index}]")
        self.model.update()


class SubProblem:
    """_summary_
    子问题实际上是一个背包问题
    """
    def __init__(self):
        self.model = gp.Model("subproblem")
        self.vars = {}
        self.constr = None
        
    def setup(self, stock_length, lengths, duals):
        self.vars = self.model.addVars(len(lengths), obj=duals, vtype=GRB.INTEGER,
                                       name="Frequency")
        self.constr = self.model.addConstr(self.vars.prod(lengths) <= stock_length,
                                           name="Knapsack")
        self.model.modelSense = GRB.MAXIMIZE
        self.model.params.outputFlag = 0
        # Stop the subproblem routine as soon as the objective's best bound becomes
        #less than or equal to one, as this implies a non-negative reduced cost for
        #the entering column.
        self.model.params.bestBdStop = 1
        self.model.update()
        
    def update(self, duals):
        self.model.setAttr("obj", self.vars, duals)
        self.model.update()


class CuttingStock:
    def __init__(self, supplies, pieces):
        self.stock_pieces, self.stock_lengths, self.stock_cost = gp.multidict(supplies) # 解析原料钢管的数据
        self.pieces, self.lengths, self.demand = gp.multidict(pieces) # multidict 每个键都应映射到值列表。
        # pieces: 需求k的编号
        # lengths: 需求k的钢管的长度
        # demand: 需求k的钢管的数量
        self.patterns = None # 所有的模式
        self.duals = [0]*len(self.pieces) # 对偶成本 数量 == 约束的个数
        piece_reqs = [length*req for length, req in pieces.values()] # 每个需求的总长度（长度 x 数量）
        # self.min_rolls = np.ceil(np.sum(piece_reqs)/stock_length) # 
        self.solution = {}
        self.master = MasterProblem()
        self.subproblem = [SubProblem() for _ in range(len(self.stock_pieces))]
        
    def _initialize_patterns(self):
        # 贪心地进行初始化：用每一根原料切每一种产品，依此作为切割方式
        patterns = []
        objcoefs = []
        for i, stock_length_sub in self.stock_lengths.items(): 
        # 要多增加一个循环
            for idx, length in self.lengths.items():
                pattern = [0]*len(self.pieces)
                pattern[idx] = stock_length_sub // length
                patterns.append(pattern)
                objcoefs.append(self.stock_cost[i])
        self.patterns = patterns
        self.master.objCoef = objcoefs
        
    def _generate_patterns(self):
        self._initialize_patterns()
        self.master.setup(self.patterns, self.demand)
        for i in range(len(self.stock_pieces)):
            self.subproblem[i].setup(self.stock_lengths[i], self.lengths, self.duals)
        while True:
            self.master.model.optimize()
            self.duals = self.master.model.getAttr("pi", self.master.constrs) # 当前主问题的dual Price
            temp_pattern = [0] * len(self.pieces)
            temp_reducedCost = 99999
            temp_objcoef = 0
            stop_iter = True
            for i  in range(len(self.stock_pieces)):
                # 从每个子问题里找到Reduced cost 最小的那个 每次只把提升最大的这一列加进去
                # 也可以把每个Reduced cost < 0的都加进去
                self.subproblem[i].update(self.duals)
                self.subproblem[i].model.optimize()
                reduced_cost = self.stock_cost[i]  - self.subproblem[i].model.objVal
                if reduced_cost < 0:
                    stop_iter = False
                    if reduced_cost < temp_reducedCost:
                        temp_reducedCost = reduced_cost
                        pattern = [0]*len(self.pieces)
                        for piece, var in self.subproblem[i].vars.items():
                            if var.x > 0.5:
                                pattern[piece] = round(var.x)
                        temp_pattern = pattern
                        temp_objcoef = self.stock_cost[i]
            
            if stop_iter:
                # 如果每个子问题的检验数都是不小于0 的 说明当前已经找到最优解了
                break
            else:
                # 否则更新主问题的结构
                self.master.update(temp_pattern, len(self.patterns), temp_objcoef)
                self.patterns.append(temp_pattern)
    def solve(self):
        self._generate_patterns()
        self.master.model.setAttr("vType", self.master.vars, GRB.INTEGER)
        self.master.model.optimize()
        for pattern, var in self.master.vars.items():
            if var.x > 0.5:
                # 对于CSP问题，可以直接
                self.solution[pattern] = round(var.x)

if __name__ == "__main__": 

    supplies = {
        0: [27, 500],
        1: [42, 900],
        2: [48, 1000],
        3: [57, 1400],
        4: [120, 3200],
        5: [150, 3900],
    }
    orders = {0:[4, 100],
            1:[5,90],
            2:[7,140],
            3:[6,42],
            4:[9,90],
            5:[17,55], 
            6:[21,53], 
            7:[25, 70],
            8:[28, 29],
            9:[31, 45],
            10:[33, 13],
            11:[36, 22],
            12:[38, 16]}

    mycsp = CuttingStock(supplies, orders)
    mycsp.solve()
    # print(mycsp.solution)
    print(f"总成本{mycsp.master.model.ObjVal}")
    print(f"总列数(决策变量) {len(mycsp.master.model.getVars())}")
    for sol in mycsp.solution:
        print(f"{mycsp.patterns[sol] } 产量 : {mycsp.solution[sol]}")

总成本215400.0
总列数(决策变量) 98
[0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0] 产量 : 8
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0] 产量 : 70
[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0] 产量 : 45
[2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0] 产量 : 13
[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] 产量 : 1
[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] 产量 : 21
[5, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 产量 : 1
[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] 产量 : 15
[0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0] 产量 : 10
[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0] 产量 : 42
[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0] 产量 : 11
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] 产量 : 1
[1, 0, 2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0] 产量 : 39
[0, 4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0] 产量 : 2
[1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0] 产量 : 27


In [1]:
import random
import gurobipy as gp
from gurobipy import GRB

# ===================== USING COLUMN GENERATION ALGORITHM ====================
import numpy as np



class MasterProblem:
    def __init__(self):
        self.model = gp.Model("master")
        self.vars = None
        self.constrs = None
        self.objCoef = None
        
    def setup(self, patterns, demand):
        num_patterns = len(patterns)
        self.vars = self.model.addVars(num_patterns, obj=self.objCoef, name="Pattern")
        self.constrs = self.model.addConstrs((gp.quicksum(patterns[pattern][piece]*self.vars[pattern]
                                                          for pattern in range(num_patterns))
                                              >= demand[piece] for piece in demand.keys()),
                                             name="Demand")
        self.model.modelSense = GRB.MINIMIZE
        self.model.params.outputFlag = 0
        self.model.update()
        
    def update(self, pattern, index, objcoef):
        new_col = gp.Column(coeffs=pattern, constrs=self.constrs.values())
        self.vars[index] = self.model.addVar(obj=objcoef, column=new_col,
                                             name=f"Pattern[{index}]")
        self.model.update()


class SubProblem:
    """_summary_
    子问题实际上是一个背包问题
    """
    def __init__(self):
        self.model = gp.Model("subproblem")
        self.vars = {}
        self.constr = None
        
    def setup(self, stock_length, lengths, duals):
        self.vars = self.model.addVars(len(lengths), obj=duals, vtype=GRB.INTEGER,
                                       name="Frequency")
        self.constr = self.model.addConstr(self.vars.prod(lengths) <= stock_length,
                                           name="Knapsack")
        self.model.modelSense = GRB.MAXIMIZE
        self.model.params.outputFlag = 0
        # Stop the subproblem routine as soon as the objective's best bound becomes
        #less than or equal to one, as this implies a non-negative reduced cost for
        #the entering column.
        self.model.params.bestBdStop = 1
        self.model.update()
        
    def update(self, duals):
        self.model.setAttr("obj", self.vars, duals)
        self.model.update()


class CuttingStock:
    def __init__(self, supplies, pieces):
        self.stock_pieces, self.stock_lengths, self.stock_cost = gp.multidict(supplies) # 解析原料钢管的数据
        self.pieces, self.lengths, self.demand = gp.multidict(pieces) # multidict 每个键都应映射到值列表。
        # pieces: 需求k的编号
        # lengths: 需求k的钢管的长度
        # demand: 需求k的钢管的数量
        self.patterns = None # 所有的模式
        self.duals = [0]*len(self.pieces) # 对偶成本 数量 == 约束的个数
        piece_reqs = [length*req for length, req in pieces.values()] # 每个需求的总长度（长度 x 数量）
        # self.min_rolls = np.ceil(np.sum(piece_reqs)/stock_length) # 
        self.solution = {}
        self.master = MasterProblem()
        self.subproblem = [SubProblem() for _ in range(len(self.stock_pieces))]
        
    def _initialize_patterns(self):
        # 贪心地进行初始化：用每一根原料切每一种产品，依此作为切割方式
        patterns = []
        objcoefs = []
        for i, stock_length_sub in self.stock_lengths.items(): 
        # 要多增加一个循环
            for idx, length in self.lengths.items():
                pattern = [0]*len(self.pieces)
                pattern[idx] = stock_length_sub // length
                patterns.append(pattern)
                objcoefs.append(self.stock_cost[i])
        self.patterns = patterns
        self.master.objCoef = objcoefs
        
    def _generate_patterns(self):
        self._initialize_patterns()
        self.master.setup(self.patterns, self.demand)
        for i in range(len(self.stock_pieces)):
            self.subproblem[i].setup(self.stock_lengths[i], self.lengths, self.duals)
        while True:
            self.master.model.optimize()
            self.duals = self.master.model.getAttr("pi", self.master.constrs) # 当前主问题的dual Price
            temp_pattern = [0] * len(self.pieces)
            temp_reducedCost = 99999
            temp_objcoef = 0
            stop_iter = True
            for i  in range(len(self.stock_pieces)):
                # 从每个子问题里找到Reduced cost 最小的那个 每次只把提升最大的这一列加进去
                # 也可以把每个Reduced cost < 0的都加进去
                self.subproblem[i].update(self.duals)
                self.subproblem[i].model.optimize()
                reduced_cost = self.stock_cost[i]  - self.subproblem[i].model.objVal
                if reduced_cost < 0:
                    stop_iter = False
                    if reduced_cost < temp_reducedCost:
                        temp_reducedCost = reduced_cost
                        pattern = [0]*len(self.pieces)
                        for piece, var in self.subproblem[i].vars.items():
                            if var.x > 0.5:
                                pattern[piece] = round(var.x)
                        temp_pattern = pattern
                        temp_objcoef = self.stock_cost[i]
            
            if stop_iter:
                # 如果每个子问题的检验数都是不小于0 的 说明当前已经找到最优解了
                break
            else:
                # 否则更新主问题的结构
                self.master.update(temp_pattern, len(self.patterns), temp_objcoef)
                self.patterns.append(temp_pattern)
    def solve(self):
        self._generate_patterns()
        self.master.model.setAttr("vType", self.master.vars, GRB.INTEGER)
        self.master.model.optimize()
        for pattern, var in self.master.vars.items():
            if var.x > 0.5:
                # 对于CSP问题，可以直接
                self.solution[pattern] = round(var.x)



if __name__ == "__main__": 
    random.seed(42)
    supplies = {
        0: [135, 500],
        1: [210, 900],
        2: [240, 1000],
        3: [335, 1400],
        4: [600, 3000],
        5: [750, 3400],
    }

    demand = dict() # 需求的量
    Demand = []
    NUM = 200
    Types = list(random.sample(range(50, 450), NUM))
    Demand = list(random.sample(range(200, 800), NUM))
    for i in range(NUM):
        demand[i] = [Types[i], Demand[i]]
    mycsp = CuttingStock(supplies, demand)
    mycsp.solve()
    # print(mycsp.solution)
    print(f"总成本: {mycsp.master.model.ObjVal}")
    print(f"总列数(决策变量) / 实际切割模块 : {len(mycsp.master.model.getVars())} / {len(mycsp.solution)}")
    for sol in mycsp.solution:
        print(f"{mycsp.patterns[sol] } 产量 : {mycsp.solution[sol]}")


Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-11
总成本: 106784700.0
总列数(决策变量) / 实际切割模块 : 1698 / 212
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 产量 : 1
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
for i in mycsp.solution:
    print(i)

### Test Instance

We now test our implementation using a toy example with 13 final pieces to be cut out from stock material units of 5,600 cm. The set of orders specifying the lengths and the demand requirements of final pieces is presented below:

| ID | Final Piece Length (cm) | Demand (units)|
|----|----|----|
| 0 | 1380 | 22 |
| 1 |  1520 | 25 |
| 2 | 1560 | 12 |
| 3 | 1710 | 14 |
| 4 | 1820 | 18 |
| 5 | 1880 | 18 |
| 6 | 1930 | 20 |
| 7 | 2000 | 10 |
| 8 | 2050 | 12 |
| 9 | 2100 | 14 |
| 10 | 2140 | 16 |
| 11 | 2150 | 18 |
| 12 | 2200 | 20 |

The patterns generated are the following:

In [17]:
mycsp.patterns

[[2, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1],
 [3, 0, 0, 0, 0],
 [0, 2, 0, 0, 0],
 [0, 0, 2, 0, 0],
 [0, 0, 0, 2, 0],
 [0, 0, 0, 0, 1],
 [4, 0, 0, 0, 0],
 [0, 3, 0, 0, 0],
 [0, 0, 2, 0, 0],
 [0, 0, 0, 2, 0],
 [0, 0, 0, 0, 1],
 [4, 0, 0, 0, 0],
 [0, 3, 0, 0, 0],
 [0, 0, 2, 0, 0],
 [0, 0, 0, 3, 0],
 [0, 0, 0, 0, 2],
 [1, 0, 0, 2, 0],
 [1, 1, 0, 0, 0],
 [0, 2, 0, 1, 0]]

The patterns that are actually used and their respective frequency are reported below:

| Pattern ID | Frequency | Description |
|----|----|----|
| 4 | 1 | 3 units of final piece 4 |
| 17 | 7 | 1 unit and 2 units of final pieces 0 and 9 |
| 19 | 2 | 1 unit of final pieces 2, 5, and 10 |
| 20 | 4 | 2 unit and 1 units of final pieces 3 and 10 |
| 22 | 1 | 2 unit and 1 units of final pieces 3 and 11 | 
| 26 | 5 | 1 unit of final pieces 1, 5, and 10 |
| 28 | 17 | 1 unit of final pieces 1, 6, and 11 |
| 29 | 4 | 1 unit and 2 units of final pieces 4 and 5 |
| 31 | 10 | 1 unit of final pieces 0, 7, and 12 |
| 32 | 3 | 1 unit of final pieces 2, 6, and 8 |
| 34 | 4 | 1 unit of final pieces 3, 4, and 8 |
| 35 | 5 | 1 unit of final pieces 0, 8, and 10 |
| 36 | 3 | 1 unit of final pieces 1, 5, and 12 |
| 38 | 7 | 1 unit of final pieces 2, 4, and 12 |

In [21]:
print(mycsp.solution)

{4: 90, 15: 45, 72: 35, 93: 1, 94: 1, 95: 1, 96: 13, 98: 28, 102: 90, 103: 2, 104: 41, 105: 15, 106: 20, 107: 10, 108: 6}


The minimum number of stock material units that must be used is:

In [5]:
print(mycsp.min_rolls)

73.0


In [19]:
print(mycsp.master.model.ObjVal)

564.0


The actual number of stock material units used by the solution found is:

In [6]:
sum(mycsp.solution.values())

73

In [10]:
orders = { key: [Types[key], Demand[key]] for key in range(100)}
# mycsp = CuttingStock(20, orders)
# mycsp.solve()
# print(mycsp.solution)
# print(mycsp.min_rolls)
# sum(mycsp.solution.values())

{3: 3, 11: 9, 13: 1, 15: 8, 30: 5, 56: 5, 58: 2, 69: 3, 70: 9, 75: 3, 76: 8, 99: 5, 113: 1, 115: 4, 117: 10, 118: 12, 119: 16, 120: 8, 122: 6, 123: 10, 124: 10, 126: 3, 127: 10, 128: 2, 129: 14, 130: 3, 135: 4, 139: 3, 152: 11, 153: 5, 158: 3, 160: 1, 161: 2, 162: 8, 163: 1, 164: 17, 166: 6, 167: 14, 168: 14, 170: 7, 171: 1, 172: 9, 173: 11, 177: 12, 179: 2, 180: 12, 181: 1, 183: 1, 184: 9, 185: 13, 186: 14, 188: 1, 189: 6, 191: 4, 194: 3, 195: 10, 196: 3, 197: 1, 198: 14, 199: 10, 200: 13, 201: 8, 202: 1, 203: 16, 204: 10, 205: 19, 206: 7, 207: 9, 208: 16, 209: 9, 210: 4, 212: 8, 213: 2, 214: 1, 215: 2, 216: 5, 217: 1, 218: 5, 221: 2, 222: 1, 226: 12, 229: 4, 231: 3, 232: 2, 233: 1, 234: 1, 235: 2, 236: 4, 240: 2, 241: 5, 244: 12, 245: 5, 246: 1, 248: 5, 249: 9, 251: 10, 252: 8, 253: 1, 256: 7, 257: 11, 258: 1, 259: 3}
648.0


651